In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, TobaccoUse
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
from functools import reduce
import json

DB Password: ········


In [21]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [22]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HT_Onset"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #Saving Cohort as Parquet: 
    df_name= (name_feature+"_"+cohort_type+"_"+frequency+"_"+str(gap_in_days)+"_gap_in_days")  
    df_final = cohort_mrn_diagnosis
    return df_final

In [26]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [30]:
#Define Features as FIBER Conditions
condition_cr = (Diagnosis("430%", "ICD-9")| Diagnosis("I60%", "ICD-10")| 
                Diagnosis("I61%", "ICD-10")| Diagnosis("I62%", "ICD-10")|
                Diagnosis("I63%", "ICD-10")| Diagnosis("I64%", "ICD-10")|
                Diagnosis("I65%", "ICD-10")| Diagnosis("431%", "ICD-9") |
                Diagnosis("432%", "ICD-9") | Diagnosis("433%", "ICD-9") |
                Diagnosis("434%", "ICD-9") | Diagnosis("435%", "ICD-9") |
                Diagnosis("436%", "ICD-9") | Diagnosis("437%", "ICD-9") |
                Diagnosis("438%", "ICD-9") | Diagnosis("I66%", "ICD-10")|
                Diagnosis("I67%", "ICD-10")| Diagnosis("I68%", "ICD-10"))
condition_Tobacco = (TobaccoUse("Yes")|
              TobaccoUse("Former"))
Anxiety_condition = (Drug('%Lorazepam%') | Drug('%Escitalopram oxalate'))

COPD_condition = (Diagnosis ("49%", "ICD-9")| Diagnosis ("50%","ICD-9")|
                  Diagnosis ("51%", "ICD-9")| Diagnosis ("J4%","ICD-10")
)
             
condition_rheumatic_heart_disease= (Diagnosis("I00%", "ICD-10")|
                           Diagnosis("I01%", "ICD-10")|
                           Diagnosis("I02%", "ICD-10")|
                                    Diagnosis("I05%", "ICD-10")|
                            Diagnosis("I06%", "ICD-10")|
                            Diagnosis("I07%", "ICD-10")|
                            Diagnosis("I08%", "ICD-10")|
                            Diagnosis("I09%", "ICD-10")|
                            Diagnosis("393%", "ICD-9")|
                           Diagnosis("394%", "ICD-9")|
                            Diagnosis("395%", "ICD-9")|
                            Diagnosis("396%", "ICD-9")|
                            Diagnosis("397%", "ICD-9")|
                           Diagnosis("398%", "ICD-9")
                                   )       
  
condition_oCV = (Diagnosis("420%", "ICD-9")| Diagnosis("I30%", "ICD-10") | 
                Diagnosis("I31%", "ICD-10")| Diagnosis("I32%", "ICD-10") |
                Diagnosis("I33%", "ICD-10")| Diagnosis("I34%", "ICD-10") |
                Diagnosis("I35%", "ICD-10")| Diagnosis("421%", "ICD-9") |
                Diagnosis("422%", "ICD-9")| Diagnosis("423%", "ICD-9") |
                Diagnosis("424%", "ICD-9") | Diagnosis("425%", "ICD-9") |
                Diagnosis("426%", "ICD-9") | Diagnosis("427%", "ICD-9") |
                Diagnosis("428%", "ICD-9") | Diagnosis("429%", "ICD-9") |
                Diagnosis("428%", "ICD-9") | Diagnosis("429%", "ICD-9") |
                Diagnosis("I36%", "ICD-10")| Diagnosis("I37%", "ICD-10") |
                Diagnosis("I38%", "ICD-10")| Diagnosis("I39%", "ICD-10") |
                Diagnosis("I40%", "ICD-10")| Diagnosis("I41%", "ICD-10") |
                Diagnosis("I42%", "ICD-10")| Diagnosis("I43%", "ICD-10") |
                Diagnosis("I44%", "ICD-10")| Diagnosis("I45%", "ICD-10") |
                Diagnosis("I46%", "ICD-10")| Diagnosis("I47%", "ICD-10") |
                Diagnosis("I48%", "ICD-10")| Diagnosis("I49%", "ICD-10") |
                Diagnosis("I50%", "ICD-10")| Diagnosis("I51%", "ICD-10") |
                Diagnosis("I52%", "ICD-10")
                )
condition_obesity= (Diagnosis("278%", "ICD-9")|
                  Diagnosis("E66%", "ICD-10"))
#new  
Med_t2d_condition = (Drug('%Metformin%') | 
                    Drug('%Blood sugar%')|
                    Drug('%Lancets%')|
                    Drug('%Glipizide%')|
                    Drug('%Insulin aspart%')|
                    Drug('%Insulin lispro%')|
                    Drug('%Glyburide%')
                    )
Med_schizo_condition = (Drug('%Aripiprazole%') | 
                            Drug('%Quetiapine %'))
Med_ld_condition = (Drug('%Simvastatin%') | 
                            Drug('%Atorvastatin%'))
Med_CV_condition = (Drug('%Lisinopril%') | 
                    Drug('%Hydrochlorothiazide%')|
                    Drug('%Metoprolol tartrate%')|
                    Drug('%Metoprolol succinate%')|
                    Drug('%Amlodipine besylate%')|
                    Drug('%Atenolol%')|
                    Drug('%Losartan potassium%')|
                    Drug('%Enalapril maleate%')
                    )
Med_depression_condition = (Drug('%Sertraline%') | 
                            Drug('%Bupropion%')| 
                            Drug('%Trazodone%')| 
                            Drug('%Citalopram%'))
condition_liver = (Diagnosis("570%", "ICD-9")|
                  Diagnosis("571%", "ICD-9")|
                  Diagnosis("572%", "ICD-9")|
                  Diagnosis("573%", "ICD-9")|
                  Diagnosis("K70%", "ICD-10")|
                  Diagnosis("K71%", "ICD-10")|
                  Diagnosis("K72%", "ICD-10")|
                  Diagnosis("K73%", "ICD-10")|
                  Diagnosis("K74%", "ICD-10")|
                  Diagnosis("K75%", "ICD-10")|
                  Diagnosis("K76%", "ICD-10")|
                  Diagnosis("K77%", "ICD-10"))
condition_lipid_metabolism = (Diagnosis("272 ", "ICD-9")|
              Diagnosis("E78.9", "ICD-10"))
condition_ihd = (Diagnosis("410%", "ICD-9")| Diagnosis("I20%", "ICD-10") | 
                Diagnosis("I21%", "ICD-10")| Diagnosis("I22%", "ICD-10") |
                Diagnosis("I23%", "ICD-10")| Diagnosis("I24%", "ICD-10") |
                Diagnosis("I25%", "ICD-10")| Diagnosis("411%", "ICD-9") |
                Diagnosis("412%", "ICD-9")| Diagnosis("413%", "ICD-9") |
                Diagnosis("414%", "ICD-9")  
                )
condition_hypersomnia = (Diagnosis("327%", "ICD-9")|
                  Diagnosis("G47%", "ICD-10"))
condition_vl = (Diagnosis("451%", "ICD-9")| Diagnosis("I80%", "ICD-10")| 
                Diagnosis("I81%", "ICD-10")| Diagnosis("I82%", "ICD-10")|
                Diagnosis("I83%", "ICD-10")| Diagnosis("I84%", "ICD-10")|
                Diagnosis("I85%", "ICD-10")| Diagnosis("452%", "ICD-9") |
                Diagnosis("453%", "ICD-9") | Diagnosis("454%", "ICD-9") |
                Diagnosis("455%", "ICD-9") | Diagnosis("456%", "ICD-9") |
                Diagnosis("457%", "ICD-9") | Diagnosis("458%", "ICD-9") |
                Diagnosis("459%", "ICD-9") | Diagnosis("I86%", "ICD-10")|
                Diagnosis("I87%", "ICD-10")| Diagnosis("I88%", "ICD-10")|
                Diagnosis("I89%", "ICD-10")
                )
condition_bp = (Measurement("%Blood Pressure%"))
features=[
    [condition_cr,"Cerebrovascular_Disease"],
    [condition_Tobacco,"Tobacco_Use"],
    [Anxiety_condition, "anxienty_med"],
    [COPD_condition, "COPD"],
    [condition_rheumatic_heart_disease,"condition_rheumatic_heart_disease"],
    [condition_oCV,"other_cardiovascular_disease"], 
    [condition_obesity, "obesity"],
    [Med_t2d_condition, "med_type_2_diabetes"],
    [Med_schizo_condition,"med_schizophrenia"],
    [Med_ld_condition,"med_lipid_disorders"],
    [Med_CV_condition,"med_cv"],
    [Med_depression_condition, "med_depression"],
    [condition_liver, "liver"],
    
  [condition_ihd,"ischemic_heart_disease"],
    [condition_hypersomnia,"hypersomnia"],
    [condition_vl,"diseases_of_veins_and_Lyphatics_and_other_diseases"],
    [condition_lipid_metabolism, "lipid_metabolism"]
       
]
 


In [27]:
x = Case_filtered_15_540.sample(100)

In [28]:
x=df_to_cohort(x)

In [31]:
x.get(condition_bp)

Hello
SELECT DISTINCT msdw_2018."D_PERSON"."MEDICAL_RECORD_NUMBER" AS "msdw_2018_D_PERSON_MEDICAL_RECORD_NUMBER" 
FROM msdw_2018."FACT" JOIN msdw_2018."D_PERSON" ON msdw_2018."FACT"."PERSON_KEY" = msdw_2018."D_PERSON"."PERSON_KEY" JOIN msdw_2018."D_UNIT_OF_MEASURE" ON msdw_2018."FACT"."UOM_KEY" = msdw_2018."D_UNIT_OF_MEASURE"."UOM_KEY" JOIN msdw_2018."B_PROCEDURE" ON msdw_2018."FACT"."PROCEDURE_GROUP_KEY" = msdw_2018."B_PROCEDURE"."PROCEDURE_GROUP_KEY" JOIN msdw_2018."FD_PROCEDURE" ON msdw_2018."FD_PROCEDURE"."PROCEDURE_KEY" = msdw_2018."B_PROCEDURE"."PROCEDURE_KEY" 
WHERE upper(msdw_2018."FD_PROCEDURE"."PROCEDURE_DESCRIPTION") LIKE ? AND msdw_2018."FACT"."NUMERIC_VALUE" IS NOT NULL
Clause starting
upper(msdw_2018."FD_PROCEDURE"."PROCEDURE_DESCRIPTION") LIKE :upper_1 AND msdw_2018."FACT"."NUMERIC_VALUE" IS NOT NULL
C
[Measurement (...)]
Measurement (...)
Fetching data for Measurement (...)


,medical_record_number,age_in_days,time_of_day_key,context_name,context_procedure_code,numeric_value,unit_of_measure
0,2252928355,22129,776,TDS,DBP,70.0,mm Hg
1,2252928355,22129,776,IBEX,DBP,70.0,mm Hg
2,2252928355,22129,776,EPIC,DBP,70.0,mm Hg
3,2252928355,22985,749,TDS,SBP,145.0,mm Hg
4,2252928355,22985,749,IBEX,SBP,145.0,mm Hg
5,2252928355,22985,749,EPIC,SBP,145.0,mm Hg
6,3014863021,26344,868,TDS,SBP,138.0,mm Hg
7,3014863021,26344,868,IBEX,SBP,138.0,mm Hg
8,3014863021,26344,868,EPIC,SBP,138.0,mm Hg
9,3757091205,19437,514,TDS,DBP,80.0,mm Hg


In [11]:
#This funtion is extracting the features for a case and control cohort. 
#input: 
#case= Pandas data frame
#control= Pandas data frame
#feature= Array of features, which contains the FIBER Condition and the name for a feature, that should be used in the data frame
#gap_in_days= Gap in days before the exposure of a patient. It has to be an integer
#frequency = type of occurence: EVER, COUNT,WINDOW
def get_features(case, control, features, gap_in_days, frequency):
    # case 
    for feature in features: 
        df_final = get_has_certain_condition(feature[0],case,feature[1],gap_in_days, frequency, "Case")
        df_final.to_parquet(df_name+".parquet")
    #control
    for feature in features: 
        df_final = get_has_certain_condition(feature[0],control,feature[1],gap_in_days, frequency, "Control")
        df_final.to_parquet(df_name+".parquet")
        

In [12]:
get_features(Case_filtered_15_540,Control_filtered_15_540,features,365,"COUNT")

Fetching data for Diagnosis (...)



Cerebrovascular_Disease_Case_COUNT_365_gap_in_days
                       number_of_encounters  Age_BP_condition  \
medical_record_number                                           
100067499                                82           24103.0   
100091035                               154           11369.0   
1004105957                               43           22961.0   
100423963                                20               NaN   
1005188482                              279           20636.0   
1006954100                               28           15242.0   
1006987597                               72           27618.0   
1007749798                               93               NaN   
1007977434                              221           25355.0   
1008498682                               87           23048.0   
1008529308                              144           19281.0   
1010016947                               32           24285.0   
1011410084                            

Fetching data for TobaccoUse (...)



Tobacco_Use_Case_COUNT_365_gap_in_days
                       number_of_encounters  Age_BP_condition  \
medical_record_number                                           
100067499                                82           24103.0   
100091035                               154           11369.0   
1004105957                               43           22961.0   
100423963                                20               NaN   
1005188482                              279           20636.0   
1006954100                               28           15242.0   
1006987597                               72           27618.0   
1007749798                               93               NaN   
1007977434                              221           25355.0   
1008498682                               87           23048.0   
1008529308                              144           19281.0   
1010016947                               32           24285.0   
1011410084                               59       

Fetching data for Drug (...)


...


hypersomnia_Control_COUNT_365_gap_in_days
                       last_encounter  number_of_encounters  first_encounter  \
medical_record_number                                                          
100002558                       21414                    15            19951   
100002884                       23929                    44            18526   
1000068212                      10116                    29             9228   
1000083464                      25630                    20            23026   
1000119948                      17087                   163            15757   
1000165222                      17506                   627            12215   
1000199143                      26941                   141            22805   
1000205924                      14867                   108            10618   
1000212584                      19220                    15            16818   
1000238281                      16497                    56            13475 

Fetching data for Diagnosis (...)



diseases_of_veins_and_Lyphatics_and_other_diseases_Control_COUNT_365_gap_in_days
                       last_encounter  number_of_encounters  first_encounter  \
medical_record_number                                                          
100002558                       21414                    15            19951   
100002884                       23929                    44            18526   
1000068212                      10116                    29             9228   
1000083464                      25630                    20            23026   
1000119948                      17087                   163            15757   
1000165222                      17506                   627            12215   
1000199143                      26941                   141            22805   
1000205924                      14867                   108            10618   
1000212584                      19220                    15            16818   
1000238281                      16497 

Fetching data for Diagnosis (...)



lipid_metabolism_Control_COUNT_365_gap_in_days
                       last_encounter  number_of_encounters  first_encounter  \
medical_record_number                                                          
100002558                       21414                    15            19951   
100002884                       23929                    44            18526   
1000068212                      10116                    29             9228   
1000083464                      25630                    20            23026   
1000119948                      17087                   163            15757   
1000165222                      17506                   627            12215   
1000199143                      26941                   141            22805   
1000205924                      14867                   108            10618   
1000212584                      19220                    15            16818   
1000238281                      16497                    56            1

In [ ]:
#copy all files to Cohort folder : 
#cp    *med_schizophrenia_Case_EVER_365* /Cohorts